## OIE.INT
- http://www.oie.int/wahis_2/public/wahid.php/Diseaseinformation/Immsummary

### 使用requests 取得列表頁面

In [1]:
import requests
url = 'http://www.oie.int/wahis_2/public/wahid.php/Diseaseinformation/Immsummary'

payload = {
'disease_type_hidden':'0',
'disease_id_hidden':'15',
'selected_disease_name_hidden':'Highly path. avian influenza (- -) ',
'disease_type':'0',
'disease_id_terrestrial':'15',
'disease_id_aquatic':'-999',
'year':'2017'
}

res = requests.post(url, data=payload)

### 使用BeautfulSoup 剖析頁面

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')

### 取得報告連結

In [15]:
alinks = soup.select('.vacborder a')

### 取得報告內頁摘要

In [63]:
#取得現在時間
from datetime import datetime
datetime.now()

datetime.datetime(2017, 8, 23, 10, 38, 40, 709239)

In [64]:
def getReportDetail(reportURL):
    res  = requests.get(reportURL)
    soup = BeautifulSoup(res.text, 'html.parser')
    country = soup.select('.Rap12-Subtitle')[0].contents[-1]
    
    # 使用prettify 可將 tag 轉換成字串
    dfs  = pandas.read_html(soup.select('.TableFoyers')[0].prettify())
    
    # 將資料轉置
    df = dfs[0].T
    # 將第一列變成欄位名
    df.columns = df.ix[0,]
    # 將第一列刪除
    df = df.drop(0)
    # 增添國家資訊
    df['country'] = country
    
    # 增添抓取時間
    df['search_time'] = datetime.now()
    return df

### 根據每個報告連結取得內文摘要

In [65]:
import re

reportURL = 'http://www.oie.int/wahis_2/public/wahid.php/Reviewreport/Review?page_refer=MapFullEventReport&reportid={}'

cnt = 0
report_ary = []
for link in alinks:
    if 'MapFullEventReport' in link['href']:
        # method 1: use split and strip
        reportid = link['href'].split(',',)[1].strip(');')
        report_ary.append(getReportDetail(reportURL.format(reportid)))
        
        # 當取得第五份報告時, 停止, 如果要取得所有報告, 就把下面程式碼拿掉
        cnt = cnt + 1
        if cnt == 5:
            break
        
        # method 2: use regular expression
        #m = re.match('javascript: open_report\("(.*?)",(\d+)\);', link['href'])
        #print(m.groups())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':


### 使用Pandas 合併報告

In [66]:
reportdf = pandas.concat(report_ary)
reportdf.columns
reportdf = reportdf[(['country', 'search_time','Causal agent', 'Date event resolved',
       'Date of confirmation of the event', 'Date of previous occurrence',
       'Date of start of the event', 'Date submitted to OIE',
       'Manifestation of disease', 'Nature of diagnosis',
       'Reason for notification', 'Related reports', 'Report date',
       'Report type', 'Serotype', 'This event pertains to'])]
reportdf.head()

,country,search_time,Causal agent,Date event resolved,Date of confirmation of the event,Date of previous occurrence,Date of start of the event,Date submitted to OIE,Manifestation of disease,Nature of diagnosis,Reason for notification,Related reports,Report date,Report type,Serotype,This event pertains to
1,Algeria,2017-08-23 10:39:26.532781,Highly pathogenic avian influenza virus,19/01/2017,30/10/2016,NaN,20/10/2016,02/11/2016,NaN,"Clinical, Laboratory (advanced)",First occurrence of a listed disease in the co...,Immediate notification (02/11/2016) Follow-up...,02/11/2016,Immediate notification,H7N1,a defined zone within the country
1,Austria,2017-08-23 10:39:32.014353,Highly pathogenic avian influenza virus,28/04/2017,09/11/2016,26/04/2006,07/11/2016,10/11/2016,Clinical disease,Laboratory (advanced),Recurrence of a listed disease,Immediate notification (10/11/2016) Follow-up...,10/11/2016,Immediate notification,H5N8,a defined zone within the country
1,Austria,2017-08-23 10:39:37.883832,Highly pathogenic avian influenza virus,20/02/2017,11/11/2016,26/04/2006,09/11/2016,11/11/2016,Clinical disease,Laboratory (advanced),Recurrence of a listed disease,Immediate notification (11/11/2016) Follow-up...,11/11/2016,Immediate notification,H5N8,a defined zone within the country
1,Bangladesh,2017-08-23 10:39:43.159502,Highly Pathogenic Avian Influenza virus,NaN,16/01/2017,04/02/2016,15/01/2017,21/01/2017,Clinical disease,"Clinical, Laboratory (basic), Laboratory (adva...",Recurrence of a listed disease,Immediate notification (21/01/2017) Follow-up...,21/01/2017,Immediate notification,H5N1,a defined zone within the country
1,Belgium,2017-08-23 10:39:48.472374,Highly pathogenic avian influenza virus,NaN,16/06/2017,28/03/2017,10/06/2017,19/06/2017,Clinical disease,"Clinical, Laboratory (advanced), Necropsy",Recurrence of a listed disease,Immediate notification (19/06/2017) Follow-up...,19/06/2017,Immediate notification,H5N8,a defined zone within the country


### 使用Pandas匯出報告

In [68]:
reportdf.to_excel('HPAI.xlsx')

### ==========以下為實驗===========

In [50]:
res  = requests.get('http://www.oie.int/wahis_2/public/wahid.php/Reviewreport/Review?page_refer=MapFullEventReport&reportid=21433')
soup = BeautifulSoup(res.text, 'html.parser')
soup.select('.Rap12-Subtitle')

[<div class="Rap12-Subtitle">Highly pathogenic avian influenza,<br/>Algeria</div>]

In [55]:
soup.select('.Rap12-Subtitle')[0].contents[-1]

'Algeria'